In [22]:
%matplotlib inline
import nibabel as nib
from nibabel.processing import resample_to_output
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import re

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import classification_report

seed = 42
np.random.seed(seed)  # for reproducibility

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [24]:
data_collector = {}
img_collector = {}
lesion_masks = glob('/wdata/mcrichter/HackUiowa2018/DATA/2018_IA_Hackathon/Factor_Data/All_subs_405_Lesion_Masks/*.nii.gz')
for les_file in lesion_masks:
    match = re.match(r'.*/([0-9]{4}).nii.gz', les_file)
    sub_id = match.groups()[0]
    les_img = nib.load(les_file)
    img_collector[sub_id] = les_img
    les_img_resamp = resample_to_output(les_img, voxel_sizes=(3, 3, 3))
    les_data = les_img_resamp.get_data()
#     les_data_1d = les_data.reshape(np.prod(les_data.shape))
    data_collector[sub_id] = les_data #les_data_1d 

In [25]:
data_collector

{'3559': array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0

In [26]:
group_df = pd.DataFrame.from_dict(data_collector, orient='index') 

ValueError: Must pass 2-d input

In [8]:
total_data = pd.read_csv('/wdata/mcrichter/HackUiowa2018/NN_behaviour/total_data_flattened.csv')
total_data.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
total_data.set_index('ID', inplace=True)

In [10]:
X = total_data.drop(["Score"], axis = 1)
y = total_data[["Score"]]

In [13]:
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
y_scaled = min_max_scaler.fit_transform(y)

# Run the normalizer on the dataframe
y_normalized = pd.DataFrame(y_scaled, columns = ['Score_normalized'])

(trainX, testX, trainY, testY) = train_test_split(X, y_normalized, test_size=0.1, random_state=seed)

In [21]:
image_shape = (182, 218, 182)
np.array(X)[1].reshape(*image_shape)
# np.reshape(X, (405, *image_shape))

ValueError: cannot reshape array of size 284456 into shape (182,218,182)